Short range lidar from 20mm to 2m (in good conditions) which plots
up the numbers onto the OLED screen of the TTGO ESP32

####  Stuff to do

Separate out the detection of device present into own file of functions

Make a closed quick enclosed loop that boots up and states I2C
devices present, waits a moment, and then states which device we have

Then it should begin dumping numbers from it

Apply to lidar, accelerometer, humidity, temperature

Place on second breadboard that has labels and wires on it.

Instructions on this

BNO055 case

Graphing case (with a jumper wire)

Exporting numbers out to wifi or to mqtt?

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
%sendtofile --source OLED_driver.py

In [2]:
from OLED_driver import i2c, fbuff, oledshow

In [3]:
print(i2c.scan())

[60, 64]


In [40]:
fbuff.fill(0)
oledshow()

In [42]:
import ustruct

def writeData(reg, val):
    val = ustruct.pack("B", val)
    i2c.writeto_mem(0x29, reg, val)

def readData(Reg, num):
    return i2c.readfrom_mem(0x29, Reg, num)

#print("Revision ID:", hex(readData(0x00c2, 1)[0]))
#print("Device ID:", hex(readData(0x00c0, 1)[0]))

def VL53L0Xinit():
    d = readData(0x0089, 1)
    d = (ord(d) & 0xFE) | 0x01
    writeData(0x0089, d)
    for r, v in [(0x88,0),(0x80,1),(0xff,1),(0,0),(0x91,0x3c),(0,1),(0xff,0),(0x80,0)]:
        writeData(r, v)
    for r, v in [(0x80,1),(0xff,1),(0,0),(0x91,0x3c),(0,1),(0xff,0),(0x80,0)]:
        writeData(r, v) 
    writeData(0x0000, 0x0002) # VL53L0X_REG_SYSRANGE_START, VL53L0X_REG_SYSRANGE_MODE_BACKTOBACK

def VL53L0Xdist():
    k = readData(0x0014, 12)
    b = ustruct.unpack(">HHH", k[6:])
    #b[0] ambientCount, b[1] signalCount
    status = k[0]>>3  # accurate when this is 11
    if status == 11:
        return b[2]/4
    return 0

In [46]:
VL53L0Xinit()

In [50]:
print(VL53L0Xdist())


41.0


In [49]:
print(readData(0x14, 12))

b'^\x06+\x00\xff\xff\x0b\x18\x00\x06\x00\xa5'


In [52]:
import time
while True:
    fbuff.fill(0)
    fbuff.text("VL53L0X short lidar", 1, 2, 1)
    fbuff.text("%.2f mm"%VL53L0Xdist(), 1, 20, 1)
    oledshow()
    time.sleep_ms(100)


...

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
KeyboardInterrupt: 


In [7]:
%%writefile VL53L0X_funcs.py

import ustruct
i2c = None

def writeData(reg, val):
    val = ustruct.pack("B", val)
    i2c.writeto_mem(0x29, reg, val)

def readData(Reg, num):
    return i2c.readfrom_mem(0x29, Reg, num)

#print("Revision ID:", hex(readData(0x00c2, 1)[0]))
#print("Device ID:", hex(readData(0x00c0, 1)[0]))

def VL53L0Xinit(li2c):
    global i2c
    i2c = li2c
    d = readData(0x0089, 1)
    d = (ord(d) & 0xFE) | 0x01
    writeData(0x0089, d)
    for r, v in [(0x88,0),(0x80,1),(0xff,1),(0,0),(0x91,0x3c),(0,1),(0xff,0),(0x80,0)]:
        writeData(r, v)
    for r, v in [(0x80,1),(0xff,1),(0,0),(0x91,0x3c),(0,1),(0xff,0),(0x80,0)]:
        writeData(r, v) 
    writeData(0x0000, 0x0002) # VL53L0X_REG_SYSRANGE_START, VL53L0X_REG_SYSRANGE_MODE_BACKTOBACK

def VL53L0Xdist():
    k = readData(0x0014, 12)
    b = ustruct.unpack(">HHH", k[6:])
    #b[0] ambientCount, b[1] signalCount
    status = k[0]>>3  # accurate when this is 11
    if status == 11:
        return b[2]/4
    return 0


Writing VL53L0X_funcs.py

